We have used a similar approach for the master and bachelor students, thus we extract the GPS values related to the masters from the GPS dataframe created above.

In [2]:
masters = []
for i in range(1,5):
    masters.append(" Master semestre {}".format(i))

In [3]:
gps_final = gps_df[gps_df['Section'] == "Informatique"][gps_df['Year'].isin(years_)][gps_df['AcPeriod'].isin(masters)]
gps_final.head()

NameError: name 'gps_df' is not defined

Once again, we have created two lists of dataframes to easily retrieve and clean the data.

In [ ]:
dfs_complete = []
dfs_simple = []
for gps_value in gps_final.index:
    payload = {'ww_x_GPS': gps_value, 'ww_i_reportModel': 133685247, 'ww_i_reportModelXsl' : 133685270}
    req = requests.get(url, params=payload)
    df = pd.read_html(req.content, header=1)[0]
    df.set_index("No Sciper", inplace=True)
    df_complete = df.rename(columns = {'Statut':'Status' + str(gps_final.loc[gps_value].AcPeriod) + ' ' + str(gps_final.loc[gps_value].Year)})
    df_simple = df.rename(columns = {'Statut':'Status' + str(gps_final.loc[gps_value].AcPeriod)})
    dfs_complete.append(df_complete)
    dfs_simple.append(df_simple)

### Assumptions
- Assumption 1 : A student must at least do the Master Semester 1 and Master Semester 2 to obtain his degree. Some students which have done their bachelor in 7 semesters have begun their master in semseter 2, so it is necessary to check if both semseters have been completed.
- Assumption 2 : If a student does a specialisation or a minor then he must completed at least 3 semesters to obtain his masters degree.

Thus, for the simple dataframe, we have kept only the students which had an entry in both fields and we discard the rest.

In [ ]:
dfMA_simple = dfs_simple[0]
for df in dfs_simple:
    dfMA_simple = dfMA_simple.combine_first(df)

In [ ]:
dfMA_complete = dfs_complete[0]
for df in dfs_complete:
    dfMA_complete = dfMA_complete.combine_first(df)

In [5]:
# Assumption 1
dfMA_simple1 = dfMA_simple[dfMA_simple['Status Master semestre 1'].notnull()]
dfMA_simple1 = dfMA_simple[dfMA_simple['Status Master semestre 2'].notnull()]
# Assumption 2
dfMA_simple2 = dfMA_simple[dfMA_simple['Mineur'].notnull()]
dfMA_simple2 = dfMA_simple[dfMA_simple['Status Master semestre 3'].notnull()]

NameError: name 'dfMA_simple' is not defined

Then using the sciper number of the students of the simple dataframe, we create a temporary dataframe containing all the raw data for the retained students of the simple dataframe. Using the same trick as before we count the values which are not null, for the columns corresponding to the Status for each semester and each year. This gives us the number of semesters needed for each student.

In [ ]:
dfMA_completetemp = dfMA_complete.loc[dfMA_simple.index]
dfMA_completetemp["Nb_Semesters"] = dfMA_complete.ix[:,8:36].notnull().sum(axis=1)

Finally, we can take a look at the mean value by gender and by specialisation.

In [ ]:
dfMA_completetemp.groupby(["Civilité"]).Nb_Semesters.mean()

In [ ]:
dfG = dfMA_completetemp.groupby(["Spécialisation"])
dfG.Nb_Semesters.mean()